In [ ]:
using ProgressMeter, PyCall, PyPlot, Cairo, Images, HDF5, MultivariateStats, Interpolations, Lasso, Distributions, ImageFiltering
using _Data
using  NaNStatistics, Statistics, Random
#using ScikitLearn
include("../project_place_cell/functions/func_map.jl")
include("../Decoding/Decoder_Functions.jl")
include("../Decoding/Decoder_Pipeline.jl")
np = pyimport("numpy")


In [ ]:
datasets_corner_cue = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 4, "jen"],
    ["20220416_160516", 6, "jen"]
];

n_datasets = length(datasets_corner_cue)

lengths = [90, 90, 90, 89, 90, 90, 90, 90, 90];
n_pos = 60

## example script to test equalizing the coverage

In [ ]:

spec_all = []
spec_all2 = []

i=1

    experiment_filename = datasets_corner_cue[i][1]
    server = datasets_corner_cue[i][2]
    experimenter = datasets_corner_cue[i][3]

    ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

    ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

    
    
C, heading, img_bg, y_fish, x_offset, x_fish, y_offset = h5open(ds, "behavior.h5"; raw = true) do file
    read(file, "C"),
    read(file, "heading"),
    read(file, "img_bg"),
    read(file, "fish_yolk_y"),
    read(file, "offset_x"),
    read(file, "fish_yolk_x"),
    read(file, "offset_y")
end;



# orientation-corrected fish location (time binned)
position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
    chamber_roi = read(position_file,"chamber_roi")
    x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
    y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
    speed_mm_s = read(position_file, "speed_mm_s")
    loc_digital = read(position_file, "loc_digital")
    x_digital = read(position_file, "x_digital")
    y_digital = read(position_file, "y_digital")
    x_bins = read(position_file, "x_bins")
    y_bins = read(position_file, "y_bins")
close(position_file)

moving_valid = speed_mm_s .> 0.1;

min_x = floor(Int64, minimum(x_fish_sweep_mean));
max_x = floor(Int64, maximum(x_fish_sweep_mean));

min_y = floor(Int64, minimum(y_fish_sweep_mean));
max_y = floor(Int64, maximum(y_fish_sweep_mean));

interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])


file = h5open(joinpath(data_path(ds_Chuyu), "neuron_spatial_info_0_$(lengths[i])_chamber_geometry_$(experiment_filename)_sigma1_n60.h5"), "r") #_whole #spatial_info_4 done on merged cells
    place_cell_index = read(file, "place_cell_index")    
    valid_neurons = HDF5.readmmap(file["valid_neurons"])
    specificity_shuffle_z = HDF5.readmmap(file["specificity_shuffle_z"])
    specificity_population_z = HDF5.readmmap(file["specificity_population_z"])
    specificity = HDF5.readmmap(file["specificity"])
    bool_index = HDF5.readmmap(file["bool_index"])
close(file)

bool_index = BitArray(bool_index)


position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
    chamber_roi = read(position_file,"chamber_roi")
    x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
    y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
    speed_mm_s = read(position_file, "speed_mm_s")
    loc_digital = read(position_file, "loc_digital")
    x_digital = read(position_file, "x_digital")
    y_digital = read(position_file, "y_digital")
    x_bins = read(position_file, "x_bins")
    y_bins = read(position_file, "y_bins")
    valid_moving_indices = read(position_file, "valid_moving_indices")
close(position_file)


file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"), "r")
    A_dFF = HDF5.readmmap(file["A_dFF"])

    z_all = HDF5.readmmap(file["Z_all"])
    centroid_x_all = HDF5.readmmap(file["X_all"])
    centroid_y_all = HDF5.readmmap(file["Y_all"])
    
    neuron_label = read(file, "neuron_label")
close(file)

n_neurons = size(A_dFF, 2)
n_sweeps = size(A_dFF, 1)
    

    # import an process OT and OB
    file = h5open(joinpath(data_path(ds_Chuyu), "region_roi_bool.h5"))
        region_names = read(file, "region_names")
        region_roi_bool = read(file, "region_roi_bool")
    close(file)


    tel_index = findall(region_names .== "Telencephalon -")[1];
    mask_tel = falses(n_neurons)

    for which_neuron in 1:n_neurons
        mask_tel[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, tel_index])
    end

    mask_tel = findall(mask_tel)

    place_cell_index = intersect(place_cell_index, mask_tel)
    

function bin_to_px(x, y, offset=true)
    if offset
        return (x .- 0.5) .* interval .+ (min_x-1), (y .- 0.5) .* interval .+ (min_y-1)
    else
        return (x .- 0.5) .* interval, (y .- 0.5) .* interval
    end
end

function px_to_bin(x, y)
    return 0.5 .+ ((x .- (min_x-1)) ./ interval), 0.5 .+ ((y .- (min_y-1)) ./ interval)
end

x_in_bins, y_in_bins = px_to_bin(x_fish_sweep_mean, y_fish_sweep_mean);

try
    mkdir(data_path(ds_Lorenz))    
catch
    println("save path exists")
end



Random.seed!(3);
neurons = rand(mask_tel, length(place_cell_index))


# maps over the whole experiment period
mean_map_all = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
count_map_all = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
@showprogress for neuron_idx in union(neurons, place_cell_index)#1:size(A_dFF, 2)

    neural_activity, which_loc = MAP.valid_activity_loc(A_dFF[:, neuron_idx], bool_index, loc_digital)
    mean_map_all[:, :, neuron_idx], count_map_all[:, :, neuron_idx], summed = MAP.calculate_map_direct(neural_activity, which_loc, n_pos; at_least_visit = 0, use_gaussian_filter=true, sigma=1, filter_mask = nothing)

end
    
    
Random.seed!(3);
start_mask = copy(valid_moving_indices)
start_mask[30*120:end] .= false
end_mask = copy(valid_moving_indices)
end_mask[1:60*120] .= false


# maps over the first and last 30min without equalizing
mean_map_all_start = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
count_map_all_start = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
@showprogress for neuron_idx in union(neurons, place_cell_index)#1:size(A_dFF, 2)
    neural_activity, which_loc = MAP.valid_activity_loc(A_dFF[:, neuron_idx], start_mask, loc_digital)
    mean_map_all_start[:, :, neuron_idx], count_map_all_start[:, :, neuron_idx], summed = MAP.calculate_map_direct(neural_activity, which_loc, n_pos; at_least_visit = 0, use_gaussian_filter=true, sigma=1, filter_mask = nothing)
end


mean_map_all_end = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
count_map_all_end = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
@showprogress for neuron_idx in union(neurons, place_cell_index)#1:size(A_dFF, 2)
    neural_activity, which_loc = MAP.valid_activity_loc(A_dFF[:, neuron_idx], end_mask, loc_digital)
    mean_map_all_end[:, :, neuron_idx], count_map_all_end[:, :, neuron_idx], summed = MAP.calculate_map_direct(neural_activity, which_loc, n_pos; at_least_visit = 0, use_gaussian_filter=true, sigma=1, filter_mask = nothing)
end




# subsampling the occupancy
for l in unique(loc_digital)
    temp = intersect(findall(loc_digital .== l), findall(valid_moving_indices))
    start_occ = temp[temp .< 30*120]
    end_occ = temp[temp .> 60*120]

    d = length(start_occ) - length(end_occ)
    if d < 0
        end_mask[sample(end_occ, -d, replace=false)] .= false
    else
        start_mask[sample(start_occ, d, replace=false)] .= false
    end
end
    



# maps over the first and last 30min with equalizing
mean_map_all_start_eq = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
count_map_all_start_eq = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
@showprogress for neuron_idx in union(neurons, place_cell_index)#1:size(A_dFF, 2)
    neural_activity, which_loc = MAP.valid_activity_loc(A_dFF[:, neuron_idx], start_mask, loc_digital)
    mean_map_all_start_eq[:, :, neuron_idx], count_map_all_start_eq[:, :, neuron_idx], summed = MAP.calculate_map_direct(neural_activity, which_loc, n_pos; at_least_visit = 0, use_gaussian_filter=true, sigma=1, filter_mask = nothing)
end


mean_map_all_end_eq = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
count_map_all_end_eq = fill(NaN32, n_pos, n_pos, size(A_dFF,2))
@showprogress for neuron_idx in union(neurons, place_cell_index)#1:size(A_dFF, 2)
    neural_activity, which_loc = MAP.valid_activity_loc(A_dFF[:, neuron_idx], end_mask, loc_digital)
    mean_map_all_end_eq[:, :, neuron_idx], count_map_all_end_eq[:, :, neuron_idx], summed = MAP.calculate_map_direct(neural_activity, which_loc, n_pos; at_least_visit = 0, use_gaussian_filter=true, sigma=1, filter_mask = nothing)
end


In [ ]:
# plot total map, map early, map late, equalized map early and equalized map late

neuron_idx = place_cell_index[10]

figure(figsize=(10,4))
subplot(1,5,1)
imshow(mean_map_all[:,:,neuron_idx]', origin="lower"); axis("off")
subplot(1,5,2)
imshow(mean_map_all_start[:,:,neuron_idx]', origin="lower"); axis("off")
subplot(1,5,3)
imshow(mean_map_all_end[:,:,neuron_idx]', origin="lower"); axis("off")
subplot(1,5,4)
imshow(mean_map_all_start_eq[:,:,neuron_idx]', origin="lower"); axis("off")
subplot(1,5,5)
imshow(mean_map_all_end_eq[:,:,neuron_idx]', origin="lower"); axis("off")